# Downloading Last.fm listening history

We'll be using this API method: http://www.last.fm/api/show/user.getRecentTracks

For anything more complicated, you might use this Python wrapper for the API: https://github.com/pylast/pylast

In [1]:
import requests, json, time, pandas as pd
from keys import lastfm_api_key as key, lastfm_user_name as username

In [2]:
pause_duration = 0.2
start_time = time.time()

In [3]:
# set up the url to connect to api
url = 'https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #you can retrieve max 200 records per call - we'll do the max
extended = 0 #retrieve extended data for each track, 0=no, 1=yes
page = 1 #page to start at

In [4]:
# make first request, just to get the total number of pages
request_url = url.format(username, key, limit, extended, page)
total_pages = requests.get(request_url).json()['recenttracks']['@attr']['totalPages']
print total_pages

1109


In [5]:
# initialize lists to contain response fields
responses = []
artist_names = []
album_names = []
track_names = []
timestamps = []

In [6]:
# request each page of data one at a time
for page in range(1, int(total_pages) + 1, 1):
    if page % 10 == 0: print page,
    time.sleep(pause_duration)
    request_url = url.format(username, key, limit, extended, page)
    responses.append(requests.get(request_url))

10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100


In [7]:
# parse the fields out of each scrobble in each page (aka response) of scrobbles
for response in responses:
    scrobbles = response.json()
    for scrobble in scrobbles['recenttracks']['track']:
        # only retain completed scrobbles (aka, with timestamp and not 'now playing')
        if 'date' in scrobble.keys():
            artist_names.append(scrobble['artist']['#text'])
            album_names.append(scrobble['album']['#text'])
            track_names.append(scrobble['name'])
            timestamps.append(scrobble['date']['uts'])

In [8]:
# create and populate a dataframe to contain the data
df = pd.DataFrame()
df['artist'] = artist_names
df['album'] = album_names
df['track'] = track_names
df['timestamp'] = timestamps
df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')

In [9]:
# save to csv and display the data
df.to_csv('data/lastfm_scrobbles.csv', index=None, encoding='utf-8')
print '{:,} total rows'.format(len(df))
df.head()

221,759 total rows


,artist,album,track,timestamp,datetime
0,George Harrison,All Things Must Pass (Remastered),Wah-Wah,1457999110,2016-03-14 23:45:10
1,Lemon Jelly,Lemon Jelly.ky,The Staunton Lick,1457998787,2016-03-14 23:39:47
2,Jens Lekman,Night Falls Over Kortedala,A Postcard to Nina,1457998486,2016-03-14 23:34:46
3,Wings,Wild Life,Love Is Strange - 1993 Digital Remaster,1457998197,2016-03-14 23:29:57
4,Jonathan Richman,"I, Jonathan",Parties In The U.S.A.,1457997913,2016-03-14 23:25:13


In [10]:
print 'Process took {} seconds to finish'.format(round(time.time() - start_time, 2))

Process took 708.98 seconds to finish
